In [ ]:
# Coleta de Preços com Selenium no Carrefour - Função por Faixa de Cidades
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime
import time
import pandas as pd
import re

def coletar_precos_carrefour(j_inicio=1, j_fim=10, termo_busca):
    data_de_hoje = datetime.date.today()
    carrefour_url = f"https://mercado.carrefour.com.br/s?q={termo_busca}"
    precos_carrefour = []

    for j in range(j_inicio, j_fim):
        print(f"\n🟦 Coletando cidade {j}...")
        service = ChromeService(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service)
        driver.get(carrefour_url)
        time.sleep(4)

        try:
            driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()
            time.sleep(2)
            driver.find_element(By.XPATH, "/html/body/div[4]/div[2]/div[2]/div/div/div[2]/div[2]/div/section/div/div[1]/button[2]").click()
            time.sleep(4)

            select = Select(driver.find_element(By.CSS_SELECTOR, "#selectCity"))
            if not select.options[j].is_enabled():
                print(f"Cidade {j} desabilitada. Pulando...")
                driver.quit()
                continue
            select.select_by_index(j)
            time.sleep(4)

            botao_cidade = driver.find_elements(By.CSS_SELECTOR, "div[class='grid grid-flow-col']")
            loja_texto = botao_cidade[0].text.strip().split("\n")
            loja = loja_texto[0].strip()
            cidade_estado = loja_texto[2].strip() if len(loja_texto) > 2 else ""
            botao_cidade[0].click()
            time.sleep(5)

            for _ in range(10):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(0.5)

            divs_carrefour = driver.find_elements(By.CSS_SELECTOR, "section[data-product-card-content='true']")
            for div in divs_carrefour:
                linhas = div.text.strip().split("\n")
                for i in range(len(linhas)):
                    nome = linhas[i]
                    if termo_busca.lower() in nome.lower():
                        for k in range(i+1, len(linhas)):
                            if re.match(r"R\$\s?\d{1,3}(\.\d{3})*,\d{2}", linhas[k]):
                                preco = linhas[k]
                                precos_carrefour.append({
                                    "produto": nome.strip(),
                                    "preco": preco.strip(),
                                    "loja": loja,
                                    "cidade/UF": cidade_estado,
                                    "data": data_de_hoje
                                })
                                break
        except Exception as e:
            print(f"Erro em cidade {j}: {e}")

        driver.quit()
        time.sleep(1)

    return pd.DataFrame(precos_carrefour)




<>:25: SyntaxWarning: invalid escape sequence '\['
<>:25: SyntaxWarning: invalid escape sequence '\['
/var/folders/hn/ff5wd5md5hz9v1n8vb5bf9f80000gn/T/ipykernel_47408/3984157459.py:25: SyntaxWarning: invalid escape sequence '\['
  driver.find_element(By.CSS_SELECTOR, "#__next > div:nth-child(5) > div.bg-white.min-h-\[39px\].border-b.border-gray-light > div > div > div").click()


In [ ]:
termo_busca="tilapia"
# Exemplo de uso para faixas maiores
#df_1 = coletar_precos_carrefour(j_inicio=1, j_fim=10)
df_1_2 = coletar_precos_carrefour(j_inicio=2, j_fim=20)
df_2 = coletar_precos_carrefour(j_inicio=20, j_fim=40)
df_3 = coletar_precos_carrefour(j_inicio=40, j_fim=60)
df_4 = coletar_precos_carrefour(j_inicio=60, j_fim=74)

# Concatenar tudo
df_total = pd.concat([df_1_2,df_2, df_3, df_4], ignore_index=True)
df_total.head()


🟦 Coletando cidade 0...
Erro em cidade 0: list index out of range

🟦 Coletando cidade 1...
Cidade 1 desabilitada. Pulando...

🟦 Coletando cidade 2...

🟦 Coletando cidade 3...

🟦 Coletando cidade 4...

🟦 Coletando cidade 5...

🟦 Coletando cidade 6...

🟦 Coletando cidade 7...

🟦 Coletando cidade 8...

🟦 Coletando cidade 9...

🟦 Coletando cidade 10...

🟦 Coletando cidade 11...

🟦 Coletando cidade 12...

🟦 Coletando cidade 13...

🟦 Coletando cidade 14...

🟦 Coletando cidade 15...

🟦 Coletando cidade 16...

🟦 Coletando cidade 17...

🟦 Coletando cidade 18...

🟦 Coletando cidade 19...

🟦 Coletando cidade 20...

🟦 Coletando cidade 21...

🟦 Coletando cidade 22...

🟦 Coletando cidade 23...
Erro em cidade 23: list index out of range

🟦 Coletando cidade 24...

🟦 Coletando cidade 25...

🟦 Coletando cidade 26...

🟦 Coletando cidade 27...

🟦 Coletando cidade 28...

🟦 Coletando cidade 29...

🟦 Coletando cidade 30...

🟦 Coletando cidade 31...

🟦 Coletando cidade 32...

🟦 Coletando cidade 33...

🟦 Cole

NameError: name 'df_1' is not defined

In [ ]:
#alterar a coluna preco de R$ 22,69 para 22.69 (float)
df_total['preco'] = df_total['preco'].str.replace('R$', '').str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
df_total['preco'] = df_total['preco'].astype(float) 

#da coluna cidade_estado separar em uma coluna 'bairro' o que estiver antes da primeira vírgula e o que estiver depois da primeira vírgula na coluna 'cidade_estado' continuará sendo cidade_estado
df_total[['bairro', 'cidade_estado']] = df_total['cidade_estado'].str.split(',', n=1, expand=True) 

df_total = df_total[['data', 'produto', 'preco', 'cidade_estado', 'bairro','loja']]


#renomear coluna cidade_estado para cidade/UF do dataframe df_total
df_total.rename(columns={'cidade_estado': 'cidade/UF'}, inplace=True)    

#substituir " - " por "/" na coluna cidade/UF do dataframe df_total
df_total['cidade/UF'] = df_total['cidade/UF'].str.replace(' - ', '/')  
df_total['cidade/UF'] = df_total['cidade/UF'].str.strip()

df_total['loja'] = df_total['loja']+df_total['bairro']


In [ ]:
df_total.to_csv(f'/Users/franciscoabraao/Documents/Jubart_2024/Painel_Precos_scrap/precos_carrefour_{termo_busca}.csv', index=False, encoding='utf-8')
